In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import nltk
import csv

## importing the necessary libraries 

In [ ]:
df = pd.read_csv('/AMAZON/dataset/train.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)

In [ ]:
df.head()

## checking the missing values

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.isna())

## imputing the title values in the missing bullet points columns

In [ ]:
df.loc[df['BULLET_POINTS'].isna(),'BULLET_POINTS'] = df[df['BULLET_POINTS'].isna()]['TITLE'] 

# dropping the description and brands columns due to presence of many missing values and no correlation between the taget column.

In [ ]:
df = df.drop(['DESCRIPTION','BRAND'],axis=1)
df.head()

In [ ]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]

## removing the special characters from the dataframe strings from the columns bullet_points and title 

TITLE

In [ ]:
df['title'] = df['TITLE'].str.lower()
df['title'] = df['title'].str.replace('\d+', '')
for char in spec_chars:
    df['title'] = df['title'].str.replace(char, ' ')
df.head()

## BULLET

In [ ]:
df['bullet'] = df['BULLET_POINTS'].str.lower()
df['bullet'] = df['bullet'].str.replace('\d+', '')
for char in spec_chars:
    df['bullet'] = df['bullet'].str.replace(char, ' ')
df.head()

## droping the original title and bullet columns 

In [ ]:
df.drop(['TITLE','BULLET_POINTS'],axis=1,inplace=True)

## STOP WORDS REMOVAL

In [ ]:
df['title'] = df['title'] + " " + df['bullet']

In [ ]:
df.head()

In [ ]:
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words

In [ ]:
def fun(text):
    if(type(text)==str):
        words = [word for word in text.split() if word not in sw_spacy]
        new_text = " ".join(words)
        return new_text
    else: 
        return text

In [ ]:
df['title'] = df['title'].astype('object').map(fun)

In [ ]:
df.head()

## imputing nan titles with the word 'cover case' due to its high presence in the dataset

In [ ]:
df.loc[df['title'].isna(),'title'] = 'cover case'

## removing the duplicate words from the dataset rows

In [ ]:
def uni(x):
    l = np.unique(x.split())
    return " ".join(l)
test['title'] = test['title'].map(uni)

## removing the non english words

In [ ]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
def fun1(text):
    if(type(text)==str):
        words1 = [word for word in text.split() if word  in words]
        new_text = " ".join(words1)
        return new_text
    else:
        return text
df['title'] = df['title'].map(fun1)
df.head()

In [ ]:
def len_fun(text):
    if(type(text)==str):
        words = text.split()
        ans = len(words)
    return ans
    else: 
        return 0

In [ ]:
df['len_title'] = df['title'].map(len_fun)

## plot for the count of words in the dataset in the title column

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x=df['len_title'])

In [ ]:
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

## STEMMING

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()
 
def stm(text):
    if(type(text)==str):
        words = text.split()
        new_text =""
        for w in words:
            new_text = new_text + " "+ ps.stem(w)
        return new_text
    else: 
        return text

df['title'] = df['title'].map(stm) 

## making a dataset with 10 samples of each browse_node_id

In [ ]:
df1 = df[df['BROWSE_NODE_ID']==0].head(10)

In [ ]:
pd.concat([df[df['BROWSE_NODE_ID']==0],df[df['BROWSE_NODE_ID']==1]],axis=0).head()

In [ ]:
df1.head()
for i in pd.unique(df['BROWSE_NODE_ID']):
    if(i!=0):
        if(df[df['BROWSE_NODE_ID']==i]['BROWSE_NODE_ID'].count()>10):
            df1 = pd.concat([df1,df[df['BROWSE_NODE_ID']==i].head(10)],axis=0)
        else:
            df1 = pd.concat([df1,df[df['BROWSE_NODE_ID']==i]],axis=0)
df1.head()

## converting the dataset into vectorizer 

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
df.dropna(axis=0,inplace=True)
tfidf_vectorizer = TfidfVectorizer()

## applying the train test split for validation
#xtrain, xval, ytrain, yval = train_test_split(df['title'], df['BROWSE_NODE_ID'], train_size=0.8, random_state=101)

xtrain_tfidf = tfidf_vectorizer.fit_transform(df['title'])

## making the sparse matrix small by using Principal component analysis

In [ ]:
from scipy.sparse import csr_matrix
sparse_dataset = csr_matrix(xtrain_tfidf)

from sklearn.decomposition import TruncatedSVD
from scipy import sparse as sp
tsvd = TruncatedSVD(n_components=500)
x_train_tsvd = tsvd.fit(sparse_dataset).transform(sparse_dataset)

sparse_datasetval = csr_matrix(xval_tfidf)
x_val_tsvd = tsvd.fit(sparse_dataset).transform(sparse_datasetval)



## modeling using the naive bayes method

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train_tsvd,ytrain)

prd = model.predict(x_val_tsvd)

from sklearn.metrics import accuracy_score
accuracy_score(prd,yval)

## due to low accuracy employing the full data as train

In [ ]:
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
from scipy.sparse import csr_matrix
sparse_dataset = csr_matrix(xtrain_tfidf)

from sklearn.decomposition import TruncatedSVD
from scipy import sparse as sp
tsvd = TruncatedSVD(n_components=5000)
xfit = tsvd.fit(sparse_dataset)
x_train_tsvd = xfit.transform(sparse_dataset)

## applying the same methods for test.csv

In [ ]:
test = pd.read_csv('/AMAZON/dataset/test.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)
test.head()

In [ ]:
## imputing the title values in the missing bullet points columns

test.loc[test['BULLET_POINTS'].isna(),'BULLET_POINTS'] = test[test['BULLET_POINTS'].isna()]['TITLE'] 

# dropping the description and brands columns due to presence of many missing values and no correlation between the taget column.

test = test.drop(['DESCRIPTION','BRAND'],axis=1)
test.head()

In [ ]:
test['title'] = test['TITLE'].str.lower()
test['title'] = test['title'].str.replace('\d+', '')
for char in spec_chars:
    test['title'] = test['title'].str.replace(char, ' ')


## BULLET

test['bullet'] = test['BULLET_POINTS'].str.lower()
test['bullet'] = test['bullet'].str.replace('\d+', '')
for char in spec_chars:
    test['bullet'] = test['bullet'].str.replace(char, ' ')


## droping the original title and bullet columns 

test.drop(['TITLE','BULLET_POINTS'],axis=1,inplace=True)

## STOP WORDS REMOVAL

test['title'] = test['title'] + " " + test['bullet']
test.drop('bullet',axis=1,inplace=True)
test['title'] = test['title'].astype('object').map(fun)



## imputing nan titles with the word 'cover case' due to its high presence in the dataset

test.loc[test['title'].isna(),'title'] = 'cover case'

## removing the duplicate words from the dataset rows
test['title'] = test['title'].map(uni)

## removing the non english words
test['title'] = test['title'].map(fun1)

test.reset_index(inplace=True)
test.drop('index',axis=1,inplace=True)

## STEMMING
test['title'] = test['title'].map(stm) 

In [ ]:
test.head()

In [ ]:
xval= test['title']
xval1 = xval[0:50000]
xval2 = xval[50000:len(xval)]
xval_tfidf = tfidf_vectorizer.transform(xval1)
sparse_datasetval = csr_matrix(xval_tfidf)
x_val_tsvd = xfit.transform(sparse_datasetval)

prd = model.predict(x_val_tsvd)

xval_tfidf = tfidf_vectorizer.transform(xval2)
sparse_datasetval = csr_matrix(xval_tfidf)
x_val_tsvd = xfit.transform(sparse_datasetval)

prd1 = model.predict(x_val_tsvd)

In [ ]:
pred = pd.concat([prd,prd1],axis=0)

In [ ]:
pred = pd.concat([pred,test['PRODUCT_ID']],axis=0)

In [ ]:
pred.to_csv('/ans.csv')